MIT License

Copyright (c) Microsoft Corporation. All rights reserved.

This notebook is adapted from Microsoft Learning mslearn-dp100 

Copyright (c) 2021 PyLadies Amsterdam, Alyona Galyeva

# Generate batch data

In [2]:
import os
import pandas as pd
import numpy as np
from azureml.core import Workspace, Dataset

In [3]:
WORKDIR = os.getcwd()

In [4]:
# Create a folder
BATCH_FOLDER = 'capstone-15-min-data'
os.makedirs(BATCH_FOLDER, exist_ok=True)
print("Folder created!")

Folder created!


In [5]:
# get the workspace from config.json
ws = Workspace.from_config()
# get the datastore to upload our data
datastore = ws.get_default_datastore()

If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


In [6]:
# retrieve test dataset from Azure Datastore
test_ds = Dataset.get_by_name(ws, name="capstone_15_min_data")
test = test_ds.to_pandas_dataframe()
test.head()

,data_index_,hour,quarter_of_day,day_of_week,is_holiday,load_actuals_mw_lag1,load_actuals_mw_lag2,load_actuals_mw_lag3,load_actuals_mw_lag4,load_actuals_mw_lag5,load_actuals_mw_lag6,f_quarter_0,f_quarter_1,f_quarter_2,f_quarter_3,f_quarter_4,f_quarter_5,is_workday
0,2020-01-01 00:00:00,1,1,2,1,95.756328,95.756328,95.756328,95.756328,95.756328,95.756328,0.065403,0.997859,0.130526,0.991445,0.19509,0.980785,False
1,2020-01-01 00:15:00,1,1,2,1,95.756328,95.756328,95.756328,95.756328,95.756328,95.756328,0.065403,0.997859,0.130526,0.991445,0.19509,0.980785,False
2,2020-01-01 00:30:00,1,1,2,1,95.756328,95.756328,95.756328,95.756328,95.756328,95.756328,0.065403,0.997859,0.130526,0.991445,0.19509,0.980785,False
3,2020-01-01 00:45:00,1,1,2,1,95.756328,95.756328,95.756328,95.756328,95.756328,95.756328,0.065403,0.997859,0.130526,0.991445,0.19509,0.980785,False
4,2020-01-01 01:00:00,2,1,2,1,95.756328,95.756328,95.756328,95.756328,95.756328,95.756328,0.065403,0.997859,0.130526,0.991445,0.19509,0.980785,False


In [11]:
X =(test.set_index(['data_index_'])
     .loc['2021-06-05 00:00:00':'2021-06-06 00:00:00']
     .reset_index(drop=True)
     # .drop(['load'], axis=1)
     .to_numpy()
     )

In [ ]:
# pick up 1 week from Monday 31-07-2017 to Sunday 06-08-2017, drop target and timestamp, convert to numpy arrow
# X = test[9480:9648]
# X = X.drop(['demand', 'timeStamp'], axis=1)
# X = X.to_numpy()

In [12]:
# Save each sample as a separate file
print("Saving files...")
for i in range(len(X)):
    fname = str(i+1) + '.csv'
    X[i].tofile(os.path.join(BATCH_FOLDER, fname), sep=",")
print("files saved!")

Saving files...
files saved!


In [13]:
datastore.upload(src_dir="capstone-15-min-data", target_path="capstone-15-min-data", overwrite=True, show_progress=True)

Uploading an estimated of 97 files
Uploading capstone-15-min-data\1.csv
Uploaded capstone-15-min-data\1.csv, 1 files out of an estimated total of 97
Uploading capstone-15-min-data\10.csv
Uploaded capstone-15-min-data\10.csv, 2 files out of an estimated total of 97
Uploading capstone-15-min-data\11.csv
Uploaded capstone-15-min-data\11.csv, 3 files out of an estimated total of 97
Uploading capstone-15-min-data\12.csv
Uploaded capstone-15-min-data\12.csv, 4 files out of an estimated total of 97
Uploading capstone-15-min-data\13.csv
Uploaded capstone-15-min-data\13.csv, 5 files out of an estimated total of 97
Uploading capstone-15-min-data\14.csv
Uploaded capstone-15-min-data\14.csv, 6 files out of an estimated total of 97
Uploading capstone-15-min-data\15.csv
Uploaded capstone-15-min-data\15.csv, 7 files out of an estimated total of 97
Uploading capstone-15-min-data\16.csv
Uploaded capstone-15-min-data\16.csv, 8 files out of an estimated total of 97
Uploading capstone-15-min-data\17.csv
U

$AZUREML_DATAREFERENCE_7c16ffc079164afe9f667caf4b80fd8d

In [14]:
# Register a dataset for the input data
batch_data_set = Dataset.File.from_files(path=(datastore, 'capstone-15-min-data/'), validate=False)
try:
    batch_data_set = batch_data_set.register(workspace=ws, 
                                             name='capstone-15-min-data',
                                             description='batch data capstone 15 forecast',
                                             create_new_version=True)
except Exception as ex:
    print(ex)

print("Done!")

Done!
